<a href="https://colab.research.google.com/github/oikn2018/CS6910_assignment_3/blob/main/sweep_w_att.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.6.0
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
! pip install wget
! pip install gdown
! pip install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=669b565bfcfd26fd71040aeeeeb10204eb6ac1d66b34a6b344dfc817801f3261
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python

In [2]:
!pip install wandb
! wandb login 519ef73bbeeba4f437e82d8aeb9cf27e62a84740

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=271a40dcca1cc23c4e6f4ff084ec805ba1fb9d03621db55baa65ef4ddb01acf7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import torch.optim as optim
from torch.autograd import Variable
import os
import gdown
from tqdm import tqdm
import wandb
from io import open
import string, time, math
import wget
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from torch.utils.data import Dataset
import re
from torchtext.data import Field, TabularDataset, BucketIterator
# import numpy as np
import spacy
# import random
# from torch.utils.tensorboard import SummaryWriter # to print to tensorboard
# from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint

In [4]:
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
# CUDA
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

In [5]:
# Getting the Dataset
url = 'https://drive.google.com/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw&export=download'

if not os.path.exists("aksharantar_sampled"):
  filename = gdown.download(url = url, quiet=False, fuzzy=True)
  print(filename)
  with ZipFile(filename, 'r') as z:
    print('Extracting files...')
    z.extractall()
    print('Done!')
  os.remove(filename)

Downloading...
From: https://drive.google.com/uc?id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw
To: /content/aksharantar_sampled.zip
100%|██████████| 14.0M/14.0M [00:00<00:00, 71.6MB/s]


aksharantar_sampled.zip
Extracting files...
Done!


In [6]:
eng_alpha = 'abcdefghijklmnopqrstuvwxyz'
pad_char = '<PAD>'

eng_alpha2idx = {pad_char: 0}
for index, alpha in enumerate(eng_alpha):
  eng_alpha2idx[alpha] = index+1

print(eng_alpha2idx)

{'<PAD>': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [50]:
# Change Indic Language here
indic_lang = 'ben'
# indic_lang = 'hin'

In [51]:
# Bengali Unicode Hex Range: 2432-2558
# Hindi Unicode Hex Range: 2304-2431

min_range = 2304
max_range = 2431

if indic_lang == 'ben':
  min_range = 2432
  max_range = 2558
elif indic_lang == 'hin':
  min_range = 2304
  max_range = 2431

indic_alpha = [chr(alpha) for alpha in range(min_range, max_range + 1)]
print(indic_alpha)
indic_alpha_size = len(indic_alpha)

indic_alpha2idx = {pad_char: 0}
for index, alpha in enumerate(indic_alpha):
  indic_alpha2idx[alpha] = index+1

print(indic_alpha2idx)

['ঀ', 'ঁ', 'ং', 'ঃ', '\u0984', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'ঌ', '\u098d', '\u098e', 'এ', 'ঐ', '\u0991', '\u0992', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', '\u09a9', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', '\u09b1', 'ল', '\u09b3', '\u09b4', '\u09b5', 'শ', 'ষ', 'স', 'হ', '\u09ba', '\u09bb', '়', 'ঽ', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ৄ', '\u09c5', '\u09c6', 'ে', 'ৈ', '\u09c9', '\u09ca', 'ো', 'ৌ', '্', 'ৎ', '\u09cf', '\u09d0', '\u09d1', '\u09d2', '\u09d3', '\u09d4', '\u09d5', '\u09d6', 'ৗ', '\u09d8', '\u09d9', '\u09da', '\u09db', 'ড়', 'ঢ়', '\u09de', 'য়', 'ৠ', 'ৡ', 'ৢ', 'ৣ', '\u09e4', '\u09e5', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯', 'ৰ', 'ৱ', '৲', '৳', '৴', '৵', '৶', '৷', '৸', '৹', '৺', '৻', 'ৼ', '৽', '৾']
{'<PAD>': 0, 'ঀ': 1, 'ঁ': 2, 'ং': 3, 'ঃ': 4, '\u0984': 5, 'অ': 6, 'আ': 7, 'ই': 8, 'ঈ': 9, 'উ': 10, 'ঊ': 11, 'ঋ': 12, 'ঌ': 13, '\u098d': 14, '\u098e': 15, 'এ': 16, 'ঐ': 17, '\u0991': 18, '\u0992': 19, 'ও': 20, 

In [52]:
indic_idx2alpha = {v: k for k, v in indic_alpha2idx.items()}
eng_idx2alpha = {v: k for k, v in eng_alpha2idx.items()}

In [53]:
def tokenize_indic(string):
  # return string.split()
  char_list =  [*string]
  char_list = [indic_alpha2idx[char] for char in char_list]
  return char_list

def tokenize_eng(string):
  # return string.split()
  char_list =  [*string]
  char_list = [eng_alpha2idx[char] for char in char_list]
  return char_list

In [54]:
# importing python package
import pandas as pd
  
file_names = ['test', 'train', 'valid']

for index, file_name in enumerate(file_names):
  # read contents of csv file
  file = pd.read_csv(f'aksharantar_sampled/{indic_lang}/{indic_lang}_{file_name}.csv')
    
  # adding header
  headerList = ['eng', f'{indic_lang}']
    
  # converting data frame to csv
  file.to_csv(f'aksharantar_sampled/{indic_lang}/{indic_lang}_{file_name}.csv', header=headerList, index=False)

In [55]:
eng = Field(sequential=True, use_vocab=True, tokenize=tokenize_eng, init_token='<sos>', eos_token='<eos>')
indic = Field(sequential=True, use_vocab=True, tokenize=tokenize_indic, init_token='<sos>', eos_token='<eos>')

In [56]:
fields={'eng': ('eng', eng), f'{indic_lang}': ('indic', indic)}

path_name = f'aksharantar_sampled/{indic_lang}'
train_name = f'{indic_lang}_train.csv'
val_name = f'{indic_lang}_valid.csv'
test_name = f'{indic_lang}_test.csv'
train_data, val_data, test_data = TabularDataset.splits(
    path= path_name,
    train=train_name,
    validation=val_name,
    test=test_name,
    format='csv',
    fields=fields
)

In [16]:
print(train_data[0].__dict__.keys())

dict_keys(['eng', 'indic'])


In [17]:
print(train_data[0].__dict__.values())

dict_values([[2, 9, 14, 4, 8, 25, 1], [45, 64, 41, 78, 39, 78, 48, 63]])


In [57]:
eng.build_vocab(train_data, max_size = 1000, min_freq = 1)
indic.build_vocab(train_data, max_size = 1000, min_freq = 1)

In [58]:
def calc_accuracy(net, device = 'cpu', data = val_data):
    # net = net.eval().to(device)
    # predictions = []
    accuracy = 0
    count = 0
    for i in range(len(data)):
        eng_word, indic_word = [j for j in data[i].__dict__.values()]
        # gt = gt_rep(indic_word, indic_alpha2idx, device)

        # outputs = infer(net, eng_word, gt.shape[0], device)
        output = translit_infer(net, eng_word, eng, indic, device, max_length=50)
        correct = 0

        for index, char in output:
          if char == indic_word[index]:
            correct += 1


        char_level_acc = correct/len(indic_word)
        
        if char_level_acc == 1.0:
          count += 1
    # print(count)
    accuracy = count/len(data)
    
    return accuracy

In [59]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=True, dropout=p)

    def forward(self, x):
        embedding = self.dropout(self.embedding(x))

        encoder_states, (hidden, cell) = self.rnn(embedding)

        hidden = (hidden[0:1] + hidden[1:2])/2
        cell = (cell[0:1] + cell[1:2])/2

        return encoder_states, hidden, cell


class Decoder(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers, p
    ):
        super(Decoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        # hidden_size*2 comes from the encoder
        self.rnn = nn.LSTM(hidden_size*2 + embedding_size, hidden_size, num_layers, dropout=p)

        # We would take the hidden states from the encoder, but we would also take the hidden state from the decoder (s(t-1), h(j))
        self.energy = nn.Linear(hidden_size*3, 1)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, encoder_states, hidden, cell):
        # x: (1, N) where N is the batch size
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        sequence_length = encoder_states.shape[0]
        # print(f'hidden 0 -> {hidden[0].shape}')
        # To add them together, we need to have the same dim along that axis
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)
        # h_reshape -> (hidden_size), encoder_states -> (hidden_size*2)
        energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim = 2)))
#         energy = self.relu(self.energy(encoder_states))

        # Compute attention values
        attention = self.softmax(energy)
        # (seq_length, N, 1) -> We'll normalize through the first dim - seq_length so that sum of attention scores equals 1 for a specific batch

        # Goal is to elementwise multiply attention with the encoder state
        attention = attention.permute(1,2,0) #-> changing order to (N,seq_length,1)
        encoder_states = encoder_states.permute(1,0,2) #-> changer order to (N, seq_length, hidden_size*2)

#         # Multiplying the above by torch.bmm we'll get: (N, 1, hidden_size*2) -> permuting it -> (1,N, hidden_size*2)
        context_vector = torch.bmm(attention, encoder_states).permute(1,0,2)


        # attention: (seq_length, N, 1), snk
        # encoder_states: (seq_length, N, hidden_size*2), snl
        # we want context_vector: (1, N, hidden_size*2), i.e knl
#         context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)

        #context_vector is for a particular timestep for decoder, since decoding one word at a time
        # we'll concat along 3rd dim to get hidden_size*3 -> concatenating attention vector and embedding input
        rnn_input = torch.cat((context_vector, embedding), dim=2)

        #Send this through RNN
        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs)

        # predictions shape: (1, N, length_target_vocabulary) to send it to
        # loss function we want it to be (N, length_target_vocabulary) so we're
        # just gonna remove the first dim
        predictions = predictions.squeeze(0)

        return predictions, hidden, cell


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(indic.vocab)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        encoder_states, hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            # print(hidden.shape, cell.shape)
            output, hidden, cell = self.decoder(x, encoder_states, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs



In [60]:
def translit_infer(model, word, eng, indic, device, max_length=50):
    tokens = tokenize_eng(word)

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, eng.init_token)
    tokens.append(eng.eos_token)


    text_to_indices = [eng.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    word_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        encoder_states, hidden, cell = model.encoder(word_tensor)

    outputs = [indic.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_char = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_char, encoder_states, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == indic.vocab.stoi["<eos>"]:
            break

    translit_res = [indic.vocab.itos[idx] for idx in outputs]

    # remove start token
    translit_res_word = ''
    translit_res = translit_res[1:]
    # return translit_res
    for i in translit_res:
      if i != "<eos>":
        translit_res_word += indic_idx2alpha[i]
      else:
        break
    return translit_res_word


In [61]:
def save_checkpoint(state, filename=f"{indic_lang}_2_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [62]:
from IPython.utils.path import target_outdated
def check_accuracy(loader, model, input_shape=None, toggle_eval=True, print_accuracy=True):
    if toggle_eval:
        model.eval()
    device = next(model.parameters()).device
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        loader.create_batches()
        for batch in loader.batches:
          for example in batch:
            num_samples += 1
            eng_word = "".join([eng_idx2alpha[val] for val in example.eng])
            indic_word = "".join([indic_idx2alpha[val2] for val2 in example.indic])
            indic_pred = translit_infer(model, eng_word, eng, indic, device, max_length=50)
            
            if indic_pred == indic_word:
              num_correct += 1

    accuracy = num_correct / num_samples
    if toggle_eval:
        model.train()
#     if print_accuracy:
#         print(f"Accuracy on validation set: {accuracy * 100:.2f}%")
    return accuracy

In [41]:
### Now model is ready to train

In [66]:
# Training Hyperparameters
num_epochs =30
learning_rate = 0.001
batch_size = 64

# Model Hyperparameters
load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(eng.vocab)
input_size_decoder = len(indic.vocab)
output_size = len(indic.vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 512
num_layers = 1
enc_dropout = 0.5
dec_dropout = 0.5

In [67]:
def train():
    
    train_iterator, val_iterator, test_iterator = BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size = batch_size,
    # Examples of similar length will be in same batch to minimize padding and save on compute
    sort_within_batch = True,
    sort_key = lambda x: len(x.eng),
    device = device)



    encoder_net = Encoder(
        input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout
        ).to(device)
    decoder_net = Decoder(
        input_size_decoder, decoder_embedding_size, hidden_size, output_size, num_layers, dec_dropout
        ).to(device)

    model = Seq2Seq(encoder_net, decoder_net).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    pad_idx = indic.vocab.stoi['<pad>']
    # if all examples in batch are of similar length, don't incur penalty for this padding
    criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

    if load_model:
      load_checkpoint(torch.load(f'{indic_lang}_checkpoint.pth.tar'), model, optimizer)
    if indic_lang == 'hin':
      word = 'bachta'
      og_translit = 'बचता'
    elif indic_lang == 'ben':
      word = 'gabhaaskar'
      og_translit = 'গাভাস্কার'
    acc_val_prev = 0
    acc_val_current = 0
    for epoch in range(num_epochs):
      print(f'Epoch [{epoch+1} / {num_epochs}]')

      checkpoint = {
          'state_dict': model.state_dict(),
          'optimizer': optimizer.state_dict()
      }
      if acc_val_current > acc_val_prev:

          if os.path.exists(f'{indic_lang}_checkpoint_new_{acc_val_prev*100:.2f}.pth.tar'):
              os.remove(f'{indic_lang}_checkpoint_new_{acc_val_prev*100:.2f}.pth.tar')
          acc_val_prev = acc_val_current
          save_checkpoint(checkpoint, f'{indic_lang}_checkpoint_new_{acc_val_current*100:.2f}.pth.tar')



      loop = tqdm(enumerate(train_iterator), total=len(train_iterator))
      for batch_idx, batch in loop:
        inp_data = batch.eng.to(device)
        target = batch.indic.to(device)

        output = model(inp_data, target)
        # output shape: (target_len, batch_size, output_dim)

        #basically reshape output keeping last output_dim same
        output = output[1:].reshape(-1, output.shape[2]) # so that first start token is not sent to out model
        # target -> (target_len, batch_size)
        target = target[1:].reshape(-1)
        optimizer.zero_grad()
        loss = criterion(output, target)

        loss.backward()

        # to avoid exploding gradients, clip them when they are above a threshold
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()

      model.eval() # turns off Dropout
      translit_res = translit_infer(model, word, eng, indic, device, max_length=50)
      print(f'Translated example word:  English: {word}, Actual: {og_translit}, Predicted: {translit_res}')
      model.train()

      print('Computing Loss and Validation Accuracy...')
      acc_val_current = check_accuracy(val_iterator, model, input_shape=None, toggle_eval=True, print_accuracy=True)
      print(f'Training Loss: {loss.item()}, Validation Accuracy: {acc_val_current * 100:.2f}%')
      print('--------------------------')



In [68]:
%%time
train()

Epoch [1 / 30]


100%|██████████| 800/800 [00:26<00:00, 30.14it/s]


Translated example word:  English: gabhaaskar, Actual: গাভাস্কার, Predicted: গাভাসকার
Computing Loss and Validation Accuracy...
Training Loss: 1.0853245258331299, Validation Accuracy: 19.93%
--------------------------
Epoch [2 / 30]
=> Saving checkpoint


100%|██████████| 800/800 [00:26<00:00, 30.28it/s]


Translated example word:  English: gabhaaskar, Actual: গাভাস্কার, Predicted: গভাসকার
Computing Loss and Validation Accuracy...
Training Loss: 0.6437900066375732, Validation Accuracy: 27.06%
--------------------------
Epoch [3 / 30]
=> Saving checkpoint


...

Translated example word:  English: gabhaaskar, Actual: গাভাস্কার, Predicted: গভাষ্কর
Computing Loss and Validation Accuracy...
Training Loss: 0.08528614789247513, Validation Accuracy: 37.19%
--------------------------
Epoch [29 / 30]


100%|██████████| 800/800 [00:26<00:00, 29.76it/s]


Translated example word:  English: gabhaaskar, Actual: গাভাস্কার, Predicted: গভাসকর
Computing Loss and Validation Accuracy...
Training Loss: 0.08410114049911499, Validation Accuracy: 36.92%
--------------------------
Epoch [30 / 30]


100%|██████████| 800/800 [00:26<00:00, 29.80it/s]


Translated example word:  English: gabhaaskar, Actual: গাভাস্কার, Predicted: গাভাসকার
Computing Loss and Validation Accuracy...
Training Loss: 0.2135738879442215, Validation Accuracy: 36.80%
--------------------------
CPU times: user 27min 7s, sys: 7.99 s, total: 27min 15s
Wall time: 27min 27s
